In [ ]:
import pandas as pd
from qgrid import show_grid
import seaborn as sns
from pandas_profiling import ProfileReport
from funpymodeling.exploratory import freq_tbl, status, profiling_num, cat_vars, num_vars
import numpy as np

### 1) Carga y preparación de datos

In [ ]:
# Fuente: https://github.com/rfordatascience/tidytuesday/blob/master/data/2020/2020-01-21/readme.md
data=pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv")

status(data)

Removemos duplicados de canciones:

In [ ]:
data=data.drop_duplicates(subset="track_id")

In [ ]:
show_grid(data, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

In [ ]:
len(data)

28356

### 2) Preparación de datos

In [ ]:
x_data=data.drop(cat_vars(data), axis=1)

# saquemos otras...
x_data=x_data.drop(['key','speechiness', 'mode', 'tempo', 'duration_ms'], axis=1)

In [ ]:
status(x_data)

### 3) Creación del modelo

Doc oficial: https://hdbscan.readthedocs.io/en/latest/

Ojo: en sklearn esta dbscan (sin la H). Es casi igual pero con mas hiperparámetros.
En HDBSCAN encuentra internamente prueba distintos valores (usen este).

In [ ]:
import hdbscan

Generamos el modelo y fiteamos:
Condición todas las variables numéricas y sin nulos.

In [ ]:
model_hdb = hdbscan.HDBSCAN()
model_hdb.fit(x_data)

HDBSCAN()

In [ ]:
# Nro de cluster
model_hdb.labels_

In [ ]:
from funpymodeling.exploratory import freq_tbl, todf

freq_tbl(model_hdb.labels_.astype('str'))

Muchos clusters! **171** 

Y uno de ellos es de ruido (indicado con el `-1`)

### 4) Tuning/regularización de HDBSCAN para menos clusters

In [ ]:
model_hdb2 = hdbscan.HDBSCAN(min_cluster_size=80).fit(x_data)

In [ ]:
freq_tbl(model_hdb2.labels_.astype('str'))

Notar el cluster de ruido. ¿qué cambió?

### 5) Análisis del modelo cluster (profiling)

In [ ]:
cluster_var='cluster' # definan nombre de var de clustering
x_data_cl=x_data.copy() # hagan copia de los datos de TR para no perderlos
x_data_cl[cluster_var]=model_hdb2.labels_ # ojo aca! el predict es de los datos normalizados

In [ ]:
x_data_cl.head()

Saquemos el cluster de ruido:

In [ ]:
x_data_cl=x_data_cl[x_data_cl['cluster']!=-1]

In [ ]:
from funpymodeling.model_validation import coord_plot
d_orig, d_transf = coord_plot(x_data_cl, cluster_var)

In [ ]:
d_orig

In [ ]:
d_transf

In [ ]:
freq_tbl(x_data_cl['cluster'])

### 6) _Sanity check!_

In [ ]:
data_check=data.copy()
data_check['cluster']=model_hdb2.labels_
#data_check=data_check[['cluster', 'track_name']]

In [ ]:
show_grid(data_check, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})

#### Calculandolo especialmente para HDBSCAN

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples

sil_avg = silhouette_score(x_data, model_hdb2.labels_)

`silhouette_score`: nos da el promedio global

In [ ]:
sil_avg

`silhouette_samples` nos da el valor silhouette para cada registro:

In [ ]:
sample_silhouette_values = silhouette_samples(x_data, model_hdb2.labels_)

In [ ]:
sample_silhouette_values

In [ ]:
res_sil=pd.DataFrame({'sil_value': sample_silhouette_values, 'cluster': model_hdb2.labels_})

In [ ]:
g = sns.FacetGrid(res_sil, col='cluster')
g = g.map(sns.kdeplot, 'sil_value')

Promedio de Silhouette por cluster:

In [ ]:
res_sil.groupby('cluster').mean('sil_value')